In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# **Data Preprocessing**

In [8]:
import pandas as pd

# Load datasets
users_df = pd.read_csv('Expanded_Users.csv')
user_history_df = pd.read_csv('Expanded_UserHistory.csv')
destinations_df = pd.read_csv('Destinations_4Travel.csv')

In [9]:
# Display the first few rows of each dataset to understand their structure
print(users_df.head())
print(user_history_df.head())
print(destinations_df.head())

   UserID   Name              Email          Preferences  Gender  \
0       1  Kavya  kavya@example.com  Beaches, Historical  Female   
1       2  Rohan  rohan@example.com    Nature, Adventure    Male   
2       3  Kavya  kavya@example.com     City, Historical  Female   
3       4  Anika  anika@example.com  Beaches, Historical  Female   
4       5  Tanvi  tanvi@example.com    Nature, Adventure  Female   

   NumberOfAdults  NumberOfChildren  
0               1                 0  
1               2                 2  
2               2                 0  
3               1                 0  
4               2                 2  
   HistoryID  UserID  DestinationID   VisitDate  ExperienceRating
0          1     525            760  2024-01-01                 3
1          2     184            532  2024-02-15                 5
2          3     897            786  2024-03-20                 2
3          4     470            660  2024-01-01                 1
4          5     989            3

In [10]:
# Merging user profiles with their history
merged_df = pd.merge(user_history_df, users_df, on='UserID')

In [11]:
# Display the first few rows of the merged dataframe
print(merged_df.head())

   HistoryID  UserID  DestinationID   VisitDate  ExperienceRating    Name  \
0          1     525            760  2024-01-01                 3  Saanvi   
1          2     184            532  2024-02-15                 5  Ishaan   
2        528     184            394  2024-03-20                 3  Ishaan   
3        676     184            878  2024-01-01                 3  Ishaan   
4        693     184            485  2024-03-20                 1  Ishaan   

                Email          Preferences  Gender  NumberOfAdults  \
0  saanvi@example.com     City, Historical  Female               2   
1  ishaan@example.com  Beaches, Historical    Male               1   
2  ishaan@example.com  Beaches, Historical    Male               1   
3  ishaan@example.com  Beaches, Historical    Male               1   
4  ishaan@example.com  Beaches, Historical    Male               1   

   NumberOfChildren  
0                 2  
1                 2  
2                 2  
3                 2  
4     

In [12]:
# Creating a pivot table to get the user-item interaction matrix
user_item_matrix = pd.crosstab(merged_df['UserID'], merged_df['DestinationID'])

# Display the first few rows of the user-item interaction matrix
print(user_item_matrix.head())

DestinationID  2     3     4     5     6     7     8     9     10    11    \
UserID                                                                      
1                 0     0     0     0     0     0     0     0     0     0   
2                 0     0     0     0     0     0     0     0     0     0   
3                 0     0     0     0     0     0     0     0     0     0   
5                 0     0     0     0     0     0     0     0     0     0   
7                 0     0     0     0     0     0     0     0     0     0   

DestinationID  ...  987   988   990   991   993   994   996   997   998   1000  
UserID         ...                                                              
1              ...     0     0     0     0     0     0     0     0     0     0  
2              ...     0     0     0     0     0     0     0     0     0     0  
3              ...     0     0     0     0     0     0     0     0     0     0  
5              ...     0     0     0     0     0     0 

# **Recommendation Site**

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Normalizing the user-item matrix
scaler = StandardScaler()
user_item_matrix_scaled = scaler.fit_transform(user_item_matrix)

# Applying Truncated SVD for dimensionality reduction
svd = TruncatedSVD(n_components=50, random_state=42)
latent_matrix = svd.fit_transform(user_item_matrix_scaled)

# Computing the similarity matrix
similarity_matrix = cosine_similarity(latent_matrix)

In [14]:
# Recommendation function
def recommend(user_id, user_features, similarity_matrix, top_n=5):
    user_idx = user_features.index.get_loc(user_id)
    similar_users = np.argsort(similarity_matrix[user_idx])[::-1]

    recommended_destinations = {}
    for sim_user in similar_users:
        for destination in user_features.columns:
            if user_features.iloc[sim_user][destination] > 0 and user_features.iloc[user_idx][destination] == 0:
                if destination not in recommended_destinations:
                    recommended_destinations[destination] = similarity_matrix[user_idx][sim_user]
                else:
                    recommended_destinations[destination] += similarity_matrix[user_idx][sim_user]

    recommended_destinations = sorted(recommended_destinations.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return [destination for destination, _ in recommended_destinations]

# Example recommendation for a user
user_id = user_item_matrix.index[0]
recommendations = recommend(user_id, user_item_matrix, similarity_matrix)
print(f"Recommended Destinations for User {user_id}:", recommendations)

Recommended Destinations for User 1: [114, 21, 250, 135, 372]


# **ChatBot(Customer Support)**

In [15]:
import nltk
from nltk.chat.util import Chat, reflections

In [16]:
# Define pairs of patterns and responses
pairs = [
    [
        r"(.*) (location|city) ?",
        ["We provide recommendations for multiple locations. Could you please specify which type of destination you are looking for?"]
    ],
    [
        r"(.*) (price|cost) ?",
        ["Our recommendations are tailored to fit various budget ranges. Could you please specify your budget?"]
    ],
    [
        r"quit",
        ["Thank you for using Travel-Ease. Have a great day!"]
    ],
]

In [17]:
# Create the chatbot
chatbot = Chat(pairs, reflections)

# Interact with the chatbot
print("Hi, how can I assist you today?")
chatbot.converse()

Hi, how can I assist you today?
>no thanks quit
None
>quit
Thank you for using Travel-Ease. Have a great day!


# **Prototype Validation**

In [18]:
# Function to validate recommendations (simplified example)
def validate_recommendations():
    test_users = user_item_matrix.index[:5]  # Select a small set of test users
    for user_id in test_users:
        recommendations = recommend(user_id, user_item_matrix, similarity_matrix)
        print(f"Recommended Destinations for User {user_id}:", recommendations)

In [19]:
# Validate the recommendations
validate_recommendations()

# Example interaction with the chatbot
print("Hi, how can I assist you today?")
chatbot.converse()

Recommended Destinations for User 1: [114, 21, 250, 135, 372]
Recommended Destinations for User 2: [952, 135, 315, 113, 978]
Recommended Destinations for User 3: [872, 732, 239, 661, 259]
Recommended Destinations for User 5: [555, 135, 723, 389, 346]
Recommended Destinations for User 7: [748, 366, 230, 7, 531]
Hi, how can I assist you today?
>bye quit
None
>quit
Thank you for using Travel-Ease. Have a great day!
